In [61]:
import pandas as pd
import datetime
from datetime import date
from collections import namedtuple
import datetimerange
from datetimerange import DateTimeRange

In [110]:
train = pd.read_csv('./dataset/observations_train.csv')

In [11]:
train.head()

,series_id,date,value
0,AAA10Y,2000-01-03 00:00:00.0000000,1.17
1,AAA10Y,2000-01-04 00:00:00.0000000,1.20
2,AAA10Y,2000-01-05 00:00:00.0000000,1.16
3,AAA10Y,2000-01-06 00:00:00.0000000,1.15
4,AAA10Y,2000-01-07 00:00:00.0000000,1.17


In [12]:
test = pd.read_csv('./dataset/observations_test.csv')

In [13]:
test.head()

,series_id,date,value
0,AAA10Y,2018-01-02 00:00:00.0000000,1.06
1,AAA10Y,2018-01-03 00:00:00.0000000,1.06
2,AAA10Y,2018-01-04 00:00:00.0000000,1.01
3,AAA10Y,2018-01-05 00:00:00.0000000,1.03
4,AAA10Y,2018-01-08 00:00:00.0000000,1.01


In [23]:
series = pd.read_csv('./dataset/series.csv')

In [25]:
daily = series[series['frequency']=='Daily']['series_id']

In [112]:
train.dtypes

series_id     object
date          object
value        float64
dtype: object

In [117]:
# timestamp convert to datetime
train['date_t']=pd.to_datetime(train['date']).apply(lambda x: datetime.datetime(x.date().year, x.date().month, x.date().day))

In [118]:
train.head()

,series_id,date,value,date_t
0,AAA10Y,2000-01-03 00:00:00.0000000,1.17,2000-01-03
1,AAA10Y,2000-01-04 00:00:00.0000000,1.20,2000-01-04
2,AAA10Y,2000-01-05 00:00:00.0000000,1.16,2000-01-05
3,AAA10Y,2000-01-06 00:00:00.0000000,1.15,2000-01-06
4,AAA10Y,2000-01-07 00:00:00.0000000,1.17,2000-01-07


In [100]:
def find_overlap_date(range1, range2):
    '''
    Calculate the overlapped time range between two ranges.
    
    range1: pd.Series, dtype=str
    range2: pd.Series, dtype=str
    return: DateTimeRange
    '''
    if type(range1)!=datetimerange.DateTimeRange:
        range1 = DateTimeRange(range1.iloc[0], range1.iloc[-1])
    if type(range2)!=datetimerange.DateTimeRange:    
        range2 = DateTimeRange(range2.iloc[0], range2.iloc[-1])
    return range1.intersection(range2)

def get_overlap(dataset, daily, sp500_range):
    '''
    get overlapped time range between sp500 and all other daily-measured series.
    
    daily: daily-measured series id set
    sp500_range: pd.series
    return: time_range
            ignored series(no overlap with sp500)
    
    '''
    overlap = sp500_range
    ignore=set()
    for series in daily:
        if series=='SP500':
            continue
        # there are series with no data!
        if len(train[train['series_id']==series])==0:
            ignore.add(series)
            continue

        range2=dataset[dataset['series_id']==series]['date']
        temp = find_overlap_date(overlap, range2)
#         print(temp)
        if str(temp)=='NaT - NaT':
            ignore.add(series)
        else:
            overlap = temp
            
            
    overlap.start_time_format = "%Y-%m-%d"
    overlap.end_time_format = "%Y-%m-%d"
    
    return overlap, ignore

In [124]:
sp=train[train['series_id']=='SP500']
overlap, ignore = get_overlap(train, daily, sp['date'])

In [125]:
overlap

2016-03-01 - 2017-12-29

In [126]:
ignore

{'DEXJPUS', 'SOFR', 'SOFRVOL'}

In [129]:
sp500_train = sp[sp['date_t'].isin(overlap.range(datetime.timedelta(days=1)))]['value']

In [130]:
len(sp500_train)

479